In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime

2024-05-01 12:32:08.297933: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 12:32:08.766538: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

tf.config.experimental.set_visible_devices(gpus[1], 'GPU')

In [3]:
BATCH_SIZE = 512
EPOCHS = 300

initial_learning_rate = 0.001
decay_steps = 1000  # 每隔多少个steps衰减一次
decay_rate = 0.96  # 学习率衰减因子

In [4]:
AS_dataset = pd.read_csv('./hairpin_filter_dataset.csv', encoding='utf-8')

In [5]:
selected_AS_dataset = AS_dataset[(AS_dataset['freq'] >= 2.5) & (AS_dataset['freq'] <= 6.5)].reset_index(drop=True)

In [6]:
full_X = selected_AS_dataset.loc[:,'freq':'S1'].to_numpy(dtype = np.float32)
full_y = selected_AS_dataset.loc[:,'S21r':'S21i'].to_numpy(dtype = np.float32)

In [7]:
# 归一化
# full_X[0] = (full_X[0] - 0.1) / (9.0 - 0.1)
full_X[:, 1] = (full_X[:, 1] - 200) / (2000 - 200)
full_X[:, 2] = (full_X[:, 2] - 200) / (2000 - 200)
full_X[:, 3] = (full_X[:, 3] - 200) / (2000 - 200)
full_X[:, 4] = (full_X[:, 4] - 200) / (4500 - 200)
full_X[:, 5] = (full_X[:, 5] - 200) / (4500 - 200)
full_X[:, 6] = (full_X[:, 6] - 200) / (4500 - 200)
full_X[:, 7] = (full_X[:, 7] - 200) / (4500 - 200)
full_X[:, 8] = (full_X[:, 8] - 9000) / (12000 - 9000)
full_X[:, 9] = (full_X[:, 9] - 100) / (300 - 100)

In [8]:
X_train, X_vali, y_train, y_vali = train_test_split(full_X, full_y, test_size=0.05, random_state=0)

In [9]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset_train = dataset_train.shuffle(buffer_size=X_train.shape[0])
dataset_train = dataset_train.batch(BATCH_SIZE)
dataset_train = dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

2024-05-01 12:32:10.642079: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9285 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [10]:
dataset_vali = tf.data.Dataset.from_tensor_slices((X_vali, y_vali))
dataset_vali = dataset_vali.shuffle(buffer_size=X_vali.shape[0])
dataset_vali = dataset_vali.batch(BATCH_SIZE)
dataset_vali = dataset_vali.prefetch(tf.data.experimental.AUTOTUNE)

In [11]:
class R2Score(tf.keras.metrics.Metric):
    def __init__(self, name='r2_score', **kwargs):
        super(R2Score, self).__init__(name=name, **kwargs)
        self.squared_errors = self.add_weight(name='squared_errors', initializer='zeros')
        self.total_squared_errors = self.add_weight(name='total_squared_errors', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        if sample_weight is not None:
            y_true = tf.multiply(y_true, tf.cast(sample_weight, tf.float32))
            y_pred = tf.multiply(y_pred, tf.cast(sample_weight, tf.float32))

        SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
        SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))

        self.squared_errors.assign(SS_res)
        self.total_squared_errors.assign(SS_tot)

    def result(self):
        return 1 - self.squared_errors / (self.total_squared_errors + tf.keras.backend.epsilon())

    def reset_state(self):
        self.squared_errors.assign(0.)
        self.total_squared_errors.assign(0.)

In [12]:
def build_model(modelDict):
    
    model = tf.keras.Sequential()
    for d in modelDict:
        model.add(tf.keras.layers.Dense(units=d['units'], activation=d['activation']))
        if d['withNorm']:
            model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(2))
    
    learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps,
        decay_rate,
        staircase=True  # 是否以指数方式精确衰减，默认False，若为True则每隔decay_steps学习率按decay_rate衰减
    )

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='mean_squared_error',
              metrics=[R2Score()])
    
    return model

In [13]:
NA1 = [{'units': 512, 'activation': 'leaky_relu', 'withNorm': True}, 
       {'units': 512, 'activation': 'leaky_relu', 'withNorm': False}, 
       {'units': 128, 'activation': 'leaky_relu', 'withNorm': False},
       {'units': 160, 'activation': 'leaky_relu', 'withNorm': False}]

In [14]:
# with mirrored_strategy.scope():
model = build_model(NA1)

In [15]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('bestsofar.h5', monitor='val_loss', verbose=1, save_best_only=True,
                             mode='min',
                             save_weights_only=True)

# tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

callbacks_list = [checkpoint]

In [16]:
model.fit(dataset_train,
          epochs=EPOCHS,
          verbose=1,
          validation_data=dataset_vali,
          callbacks=callbacks_list)

Epoch 1/300


2024-05-01 12:32:12.370324: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x561a78f23bc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-01 12:32:12.370374: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-05-01 12:32:12.381190: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-01 12:32:12.495021: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-05-01 12:32:12.601299: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


756/761 [============================>.] - ETA: 0s - loss: 0.0733 - r2_score: 0.6967
Epoch 1: val_loss improved from inf to 0.04330, saving model to bestsofar.h5
761/761 [==============================] - 6s 4ms/step - loss: 0.0731 - r2_score: 0.6267 - val_loss: 0.0433 - val_r2_score: 0.5375
Epoch 2/300
758/761 [============================>.] - ETA: 0s - loss: 0.0412 - r2_score: 0.7314
Epoch 2: val_loss improved from 0.04330 to 0.04179, saving model to bestsofar.h5
761/761 [==============================] - 4s 4ms/step - loss: 0.0413 - r2_score: -0.0620 - val_loss: 0.0418 - val_r2_score: 0.6509
Epoch 3/300
751/761 [============================>.] - ETA: 0s - loss: 0.0344 - r2_score: 0.4212
Epoch 3: val_loss improved from 0.04179 to 0.02353, saving model to bestsofar.h5
761/761 [==============================] - 4s 4ms/step - loss: 0.0344 - r2_score: 0.2984 - val_loss: 0.0235 - val_r2_score: 0.7575
Epoch 4/300
749/761 [============================>.] - ETA: 0s - loss: 0.0348 - r2_score

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               5632      
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 128)               65664     
                                                                 
 dense_3 (Dense)             (None, 160)               20640     
                                                                 
 dense_4 (Dense)             (None, 2)                 322       
                                                        

In [18]:
model.load_weights('bestsofar.h5')

In [19]:
model.save('fullfreq25-65_surogate.h5', include_optimizer=False)

/home/delta/miniconda3/envs/jt_ma/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
